In [1]:
## SIMPLEX for canonical form, min c^T x, s.t. Ax = b, x>= 0
## Also we assume nondegeneracy

import numpy as np
import copy

LB = 1e-4

class simplex_cano_solver:
    
    def __init__(self, A, b, c, basis = [], rule = "standard"):
        
        self.n_var = len(c)
        self.n_eq = len(b)
        self.base_finding_step = 0
        self.opti_finding_step = 0
        self.rule = rule
        
        assert len(A) == self.n_eq
        assert len(A[0]) == self.n_var
        
        self.original_A = A
        self.original_b = b
        self.original_c = c
        
        neg_idx = np.where(b < 0)[0]
        
        A[neg_idx] = -A[neg_idx]
        b[neg_idx] = -b[neg_idx]
        
        
        self.A = A
        self.b = b
        self.c = c
        
        self.basis_index = np.zeros(self.n_eq)
        self.curr_neg = 0
        
        if len(basis) == 0:
            
            self.find_basis()
            
        else:
            
            assert len(basis) == self.n_eq
                
            self.basis_index = basis
                
            self.reducefrom_basis()
            
    def get_min_val(self):
        
        return -self.curr_neg
    
    def rounding(self, lb):
        
        self.A[np.abs(self.A) < lb] = 0
        self.b[np.abs(self.b) < lb] = 0
        self.c[np.abs(self.c) < lb] = 0
            
    def reducefrom_basis(self, mini_turb = False, entering_idx = None):
        
        slice_mat = self.A[:, self.basis_index]
        
        if not mini_turb:
            
            trans_mat = np.linalg.inv(slice_mat)
            
        else:
            
            trans_mat = copy.deepcopy(slice_mat)
            
            assert sum(np.diag(slice_mat) - np.array([1.0] * self.n_eq)) < 1e-2
            
            trans_mat_idx = np.where(self.basis_index == entering_idx)[0][0]
            
            trans_mat[:,trans_mat_idx] = - trans_mat[:,trans_mat_idx]
            trans_mat[trans_mat_idx, trans_mat_idx] = 1.0
            
        
            
        self.A = trans_mat @ self.A
        self.b = trans_mat @ self.b
        
        self.curr_neg -= sum(self.b * self.c[self.basis_index])
        
        self.c = self.c - self.A.T @ self.c[self.basis_index] 
        
    def mini_replace(self, entering_idx, leaving_idx):

        
        replacehappen_row = np.where(self.basis_index == leaving_idx)[0][0]
        
        self.basis_index[replacehappen_row] = entering_idx
        
        self.b[replacehappen_row] /= self.A[replacehappen_row, entering_idx]
        
        self.A[replacehappen_row] /= self.A[replacehappen_row, entering_idx]
        
        self.reducefrom_basis(mini_turb = True, entering_idx = entering_idx)
        
        
    def solve(self, rule = None, terminate = 1000):
        
        if not rule:
            
            rule = self.rule
        
        assert np.min(self.b) > 0
        
        count = 0
        
        result_dict = {
            "degenerate" : False,
            "unbounded" : False,
            "early_terminate" : False,
            "solved" : False
        }
        
        while(count < terminate):
            
            ## rounding
            self.rounding(LB)
            
            ## search for a pivot
            if np.min(self.c) >= 0:
                result_dict["solved"] = True
                return result_dict
            
            ## pivoting
            if rule == "standard":
                
                # standard rule cannot deal with degenerate
                
                neg_idx = np.where(self.c < 0)[0]
                
                for col in neg_idx:
                    
                    if max(self.A[:, col]) <= 0:
                        
                        result_dict["unbounded"] = True
                        return result_dict
                    
                min_idx = np.where(self.c == np.min(self.c))[0]
                
                entering_idx = np.random.choice(min_idx,1)[0]
                
                ## now the leaving_idx
                
                judge = self.b / np.clip(np.abs(self.A[:, entering_idx]), 1e-6, None) * np.sign(self.A[:, entering_idx])
                min_val = np.min(judge[self.A[:, entering_idx] > 0])
                
                if min_val == 0:
                    
                    print(entering_idx)
                    
                    print(self.A)
                    print(self.b)
                    
                    result_dict['degenerate'] = True
                    return result_dict
                    
                
                leaving_idxes = np.where((self.A[:, entering_idx] > 0)&(judge == min_val) )[0]
                leaving_idx = self.basis_index[np.random.choice(leaving_idxes,1)[0]]
                
                self.mini_replace(entering_idx, leaving_idx)
                                
            
            else:
                
                print("rule error")
            
            
            
            
            count += 1
            self.opti_finding_step += 1
            
        result_dict["early_terminate"] = True
        return result_dict
        
        
        
        
    def find_basis(self):
        
        # consider a new problem to solve
        new_A = np.concatenate((self.A, np.identity(self.n_eq)), axis = 1)
        new_b = copy.deepcopy(self.b)
        
        
        new_c = np.zeros(self.n_var + self.n_eq)
        new_c[self.n_var:] = 1
        
        basis = np.where(new_c == 1)[0]
        
        new_probins = simplex_cano_solver(new_A, new_b, new_c, basis = basis)
        
        state = new_probins.solve()
        
        if not state["solved"]:
            
            print(state)
            
            raise Exception("basis not found!")
            
        opt = new_probins.get_min_val()
        
        if opt > LB:
            
            print(opt)
            
            raise Exception("problem infeasible!")
            
        
            
        
        ## there might need to be one slightly tuning step
        ## like, removing redundant equations
        candidate_idx = new_probins.basis_index
        bad_idxes = candidate_idx[candidate_idx >= self.n_var]
        
        if len(bad_idxes) > 0:
        
            assert np.min(new_probins.b[candidate_idx >= self.n_var]) == 0

            if np.min(np.abs(new_probins.A[candidate_idx >= self.n_var, :self.n_var])) == 0:

                raise Execption("problem degenerate")
                
            good_basis = candidate_idx[candidate_idx < self.n_var]
            
            good_A = new_probins.A[candidate_idx < self.n_var, :self.n_var]
            
            good_b = new_probins.b[candidate_idx < self.n_var]
            
            # now reform the original problem
            
            
            
            self.n_eq = len(good_b)
            self.base_finding_step = new_probins.opti_finding_step
        
        
            self.A = good_A
            self.b = good_b
                
            self.basis_index = good_basis

            
        else:
            
            self.base_finding_step = new_probins.opti_finding_step
            
            self.A = new_probins.A[:, :self.n_var]
            self.b = new_probins.b
            
            self.basis_index = candidate_idx
            
        self.reducefrom_basis()




In [11]:
import cvxpy as cp
import numpy as np
from cvxpy_solve import ComputeLP

# Problem data.
m = 30
n = 20
np.random.seed(200)
A = np.random.randint(1, 10, size=(m, n))
A = np.concatenate((A, np.identity(m)), axis = 1)
b = np.random.randint(1, 10, size=m)
c = np.random.randint(1, 10, size=n + m)

print(ComputeLP(A, b, c))

solver = simplex_cano_solver(A, b, c)
solver.solve()
solver.get_min_val()

(array([-1.08405017e-10, -7.06994313e-11,  2.77777776e-02, -8.42910846e-11,
        1.68998861e-09,  1.00034020e-10,  1.13888888e-01, -5.01541885e-11,
       -9.62110235e-11,  3.68725737e-11, -3.55126150e-11, -1.15614767e-10,
       -8.86586822e-11, -1.06098724e-10, -1.36844818e-11,  1.55555555e-01,
       -6.09394323e-11,  7.22222230e-02, -7.99783168e-11, -1.00906626e-10,
        4.20555555e+00,  6.63055555e+00,  2.45277778e+00,  8.04999999e+00,
        6.14166666e+00,  3.93055556e+00,  2.45277778e+00,  9.33333335e-01,
        5.63611111e+00,  1.80555555e-01, -1.53463457e-10,  4.64722223e+00,
        3.53888890e+00,  1.58231922e-10,  5.18611111e+00,  6.59444444e+00,
        6.71944444e+00,  5.06111111e+00,  2.70277778e+00,  3.30277777e+00,
       -1.66562508e-10,  5.06388888e+00,  6.95555556e+00,  5.75833333e+00,
        6.70000000e+00,  1.52749888e-09,  7.04166665e+00,  3.44444444e-01,
        3.36111110e+00,  5.09444445e+00]), 619.1638885906173)


619.1638888888889